In [ ]:
#!pip3 install aitextgen

In [ ]:
import pandas as pd
from string import punctuation
import re
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

In [ ]:
features = pd.read_csv("final_df2.csv")
features.head()

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,...,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
0,0,chicken_run,"Jun 21, 2000",G,"['animation', 'kids and family', 'comedy', 'ad...",English,['Peter Lord'],1h 25m,NaN,"This engaging stop-motion, claymation adventur...",...,0.208500,0.027125,0.14000,0.0675,0.79250,0.130605,0.127229,0.261164,0.611607,"Chicken Run, if anything, offers an alternativ..."
1,1,the_taste_of_others,"Feb 26, 2002",R,['comedy'],French (France),['Agnès Jaoui'],1h 52m,$635.3K,Castella (Jean-Pierre Bacri) is a successful b...,...,0.237250,0.062625,0.25505,0.0301,0.71490,0.400550,0.044488,0.232734,0.722778,Succeeds with believable characters and situat...
2,2,yi_yi,"May 8, 2001",NaN,['drama'],Edward Yang,['Edward Yang'],2h 53m,$969.1K,"Set in Taiwan, the film follows the lives of t...",...,0.152556,0.041444,0.20600,0.0403,0.75370,0.348930,0.169338,0.233988,0.596674,"I felt like I had lived it, and not just becau..."
3,3,memento,"Mar 1, 2017",R (Language|Drug Content|Violence),['mystery and thriller'],English,['Christopher Nolan'],1h 56m,$25.5M,Leonard (Guy Pearce) is tracking down the man ...,...,0.205429,0.054571,0.10860,0.0562,0.83515,0.157130,0.204062,0.217740,0.578198,Think of all the ways that a linear film struc...
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",...,0.289333,0.013333,0.22375,0.0243,0.75195,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...


In [ ]:
genre="documentary"
features=features.dropna(subset=["genre"])
selected_genre=features[features["genre"].str.contains(genre)]
selected_genre

,Unnamed: 0,movie,release date,rating,genre,language,directors,duration,Box Office,Description,...,positive_audience,negative_audience,positive,negative,neutral,compound,aud_asp_neutral,aud_asp_negative,aud_asp_positive,critic_reviews
4,4,the_life_and_times_of_hank_greenberg,"Jul 2, 1999",PG (Mild Language|Thematic Elements),"['biography', 'documentary']",English,['Aviva Kempner'],1h 29m,$1.7M,"""The Life and Times of Hank Greenberg"" is a hu...",...,0.289333,0.013333,0.223750,0.024300,0.751950,0.574785,0.047392,0.034910,0.917698,It's an adroit manipulation of interviews and ...
11,11,dark_days,"Dec 4, 2015",NaN,['documentary'],Marc Singer,['Marc Singer'],1h 34m,$436.8K,"""Dark Days"" is a feature length documentary ab...",...,0.221500,0.075000,0.090550,0.108050,0.801450,-0.006655,0.159434,0.424432,0.416135,This is the world discovered and illuminated b...
12,12,filth_and_the_fury,"Mar 29, 2000",R (Strong Language|Sexual Content|Drug Use),['documentary'],English,['Julien Temple'],1h 48m,$606.6K,"Assembled from unseen archive footage, rare of...",...,0.143222,0.111778,0.034091,0.000000,0.965909,0.057900,0.130965,0.411622,0.457413,"[]Best for older teens, especially Sex Pistols..."
20,20,wattstax,"Feb 18, 2016",R,"['music', 'documentary']",English,['Mel Stuart'],1h 38m,NaN,"In 1972 at the Los Angeles Coliseum, director ...",...,0.181714,0.018286,0.172250,0.037800,0.789950,0.423465,0.097406,0.077102,0.825492,All [the performers] draw lively reactions fro...
35,35,original_kings_of_comedy,"Aug 18, 2000",R (Language and Sex Related Humor),"['comedy', 'documentary']",English,['Spike Lee'],1h 55m,$38.2M,A cultural phenomenon for our time: Spike Lee ...,...,0.300200,0.050400,0.144250,0.054350,0.801400,0.317675,0.155835,0.421047,0.423119,[Spike Lee] knows the show belongs to his come...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2078,2078,the_go_gos,"Jul 31, 2020",NaN,"['music', 'documentary']",Alison Ellwood,['Alison Ellwood'],1h 38m,NaN,"The Go-Go's become the first successful, all-g...",...,0.289375,0.042125,0.160100,0.024350,0.815550,0.443645,0.088978,0.274056,0.636966,As director Alison Ellwood shows in her briskl...
2079,2079,fireball_visitors_from_darker_worlds,"Nov 13, 2020",NaN,['documentary'],"Werner Herzog, \n \n ...",['Werner Herzog'],1h 37m,NaN,A journey across the planet and universe explo...,...,0.250500,0.074500,0.146813,0.051312,0.802000,0.221150,0.009454,0.464325,0.526221,Fireball: Visitors from Darker Worlds is an ab...
2082,2082,assassins_2020,"Dec 11, 2020",NaN,['documentary'],Ryan White,['Ryan White'],1h 44m,NaN,The audacious murder of the brother of North K...,...,0.149500,0.133000,0.134400,0.136300,0.729400,-0.087650,0.154302,0.314949,0.530749,This is an extraordinary story and the thoroug...
2086,2086,feels_good_man,"Sep 4, 2020",NaN,['documentary'],Arthur Jones,['Arthur Jones'],1h 32m,NaN,An animated frog becomes a symbol of hate.,...,0.245000,0.031429,0.154600,0.040600,0.804800,0.382767,0.108789,0.248282,0.642930,Seeing what happened to Furie and his chill st...


In [ ]:
combined = ' '.join(selected_genre["critic_reviews"])
#processed_text = combined.lower()
#processed_text=processed_text.translate(str.maketrans("", "", punctuation))
processed_text = re.sub(r'[^\x00-\x7f]',r'', combined) 

In [ ]:
text_file = open("documentary.txt", "w")
text_file.write(processed_text)
text_file.close()

In [ ]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

file_name = 'documentary.txt'

train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"


config = GPT2ConfigCPU()


ai = aitextgen(tokenizer_file=tokenizer_file, config=config)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)

ai.train(data, batch_size=8, num_steps=50000, generate_every=5000, save_every=5000)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


5,000 steps reached: saving model to /trained_model
5,000 steps reached: generating sample texts.
 with the facts. Alow is a movie no much to the toneyle.It is a systemically, if it's also a bit too often horful, the film is.For the film is a commonisite a summin
10,000 steps reached: saving model to /trained_model
10,000 steps reached: generating sample texts.
 that self-explay.We see this is a real-open-bal countist important to our time.A fascinating, horrorly personal documentary.Theirth of a lively convincing and absolute and pur
15,000 steps reached: saving model to /trained_model
15,000 steps reached: generating sample texts.
, the film is a self-doving and retired.After Isurfmanned Story, it's a joyfully moving look at a public, and a complex documentary.A testhensive work of the waven
20,000 steps reached: saving model to /trained_model
20,000 steps reached: generating sample texts.
 of the rich, and scenes.Special film, Mr. Gibney, the Anwrilla and Spitzer is the film, the un

In [ ]:
ai = aitextgen(model_folder="trained_model",
                tokenizer_file="aitextgen.tokenizer.json")

ai.generate(10, prompt="the bridesmaid:")

The Bridesmaid is a well-creeped psychological thriller that's a solid film about the symbolism. [] The Babadook uses the final provides are more ever before these makings of the
The Bridesmaid is a calling card for a folk into the frankness of the future.Hollywood is so good.Even in the air, this film of a women, a comic.The movie wis
The Bridesmaid is an engaging, and with original exciting enough critique of the real-star performance.The Lighthouse of the film is a singular and disturbing, rigorously exciting. His
The Bridesmaiden are nothing more than the past. If it was being a better time time.All Is Lost is a good deal of a life, but how quickly their pain and creepy plot twists, it's cons
The Bridesmaid is like Elly was in creating a study of this movie.A fascinating popular breathless. A brainy is the film's refreshing, but also another is monster in a
The Bridesmaid is a fresh instead of a female palpable-suities.The film's latest film is a pulse-film meditary, and it's not a